In [ ]:
import matplotlib.pyplot as plt
#from datapackage import Package
import pandas as pd
import numpy as np
import geopandas
import plotly.express as px
import time
from scipy.stats import wilcoxon


import os
parentDirectory = os.path.abspath(os.path.join(os.path.join(os.getcwd(), os.pardir), os.pardir))
DATA_DIR = parentDirectory +'/data/'
FIGURES_DIR = parentDirectory +'/figures/'
%matplotlib inline

In [ ]:
'''
package = Package('https://datahub.io/core/country-list/datapackage.json')
from pycountry_convert import country_alpha2_to_country_name, country_name_to_country_alpha3

for resource in package.resources:
    if resource.descriptor['datahub']['type'] == 'derived/csv':
        df = geopandas.GeoDataFrame(resource.read())
 
df.columns = ['country','code']

df['country_alpha_3'] = df.code.apply(lambda x: country_name_to_country_alpha3(country_alpha2_to_country_name(x)))

from pytrends.request import TrendReq

pytrends = TrendReq(hl='en-US', tz=360)


list_entries = []

for _,row in df.iterrows():
    try:
        print(row['code'])

        entry = {}
        #collect recipe
        pytrends.build_payload(kw_list=['/m/0p57p'], cat =0,geo = row['code'], timeframe='2019-01-01 2020-12-31')
        resp = pytrends.interest_over_time()
        resp = resp.reset_index()
        resp = resp.drop(['isPartial'], axis = 1)

        entry['country'] = row['country']
        entry['code2'] = row['code'] 
        entry['code3'] = row['country_alpha_3'] 
        entry['series'] = 'recipie'
        entry['values'] = resp['/m/0p57p'].values
        list_entries.append(entry)


        entry = {}
        #collect restaurant
        pytrends.build_payload(kw_list=['/m/06l8d'], cat =0,geo = row['code'], timeframe='2019-01-01 2020-12-31')
        resp = pytrends.interest_over_time()
        resp = resp.reset_index()
        resp = resp.drop(['isPartial'], axis = 1)

        entry['country'] = row['country']
        entry['code2'] = row['code'] 
        entry['code3'] = row['country_alpha_3'] 
        entry['series'] = 'restaurant'
        entry['values'] = resp['/m/06l8d'].values
        list_entries.append(entry)

        time.sleep(3)
    except:
        continue
        
df_to_plt = pd.DataFrame(list_entries)

df_to_plt.to_pickle("dataframe_map.pckl")
''';

In [ ]:
def calculate_p_val(x):
    if len(x)==52*2:
        return wilcoxon(x[52:] - x[:52])[1]
    else:
        return 1

df_to_plt = pd.read_pickle(DATA_DIR + "dataframe_map.pickle")
df_to_plt['change_2020'] = df_to_plt['values'].apply(lambda x: (np.sum(x[52:]) \
                                                                - np.sum(x[:52]))/np.sum(x[:52]))

df_to_plt['p_value'] = df_to_plt['values'].apply(lambda x: calculate_p_val(x))

In [ ]:
df_to_plt['Percentage change'] = df_to_plt["change_2020"]

In [ ]:
fig = px.choropleth(df_to_plt.loc[(df_to_plt['series']=='recipie') & \
                                  (df_to_plt['p_value']<0.05)], locations="code3", 
                    locationmode="ISO-3", color="Percentage change", 
                    hover_name="country", range_color=[-0.75,0.75], 
                    color_continuous_scale="RdBu",width = 900, height = 305)

fig.update(layout_coloraxis_showscale=True, layout_title_text='')
fig.update_layout(
                 margin_r = 0,margin_l = 5,margin_b = 0,margin_t = 0,# customize legend orientation & positio
    )
fig.update_geos(
    resolution=110,
    showcoastlines=True, coastlinecolor="Black",countrywidth = 0.1,
    showland=True, landcolor="Gray",
    showocean=False, oceancolor="LightBlue", countrycolor = "Black",
    #showlakes=True, lakecolor="Blue",
    #showrivers=True, rivercolor="Blue"
    center = {'lat': 30,'lon':0}
)
    

fig.write_image(FIGURES_DIR + 'recipe_map.pdf')

In [ ]:
fig = px.choropleth(df_to_plt.loc[(df_to_plt['series']=='restaurant') & \
                                  (df_to_plt['p_value']<0.05)], locations="code3", 
                    locationmode="ISO-3", color="Percentage change", 
                    hover_name="country", range_color=[-0.75,0.75], 
                    color_continuous_scale="RdBu",width = 900, height = 305)

fig.update(layout_coloraxis_showscale=True, layout_title_text='')
fig.update_layout(
                 margin_r = 0,margin_l = 5,margin_b = 0,margin_t = 0,# customize legend orientation & positio
    )
fig.update_geos(
    resolution=110,
    showcoastlines=True, coastlinecolor="Black",
    showland=True, landcolor="Gray",
    showocean=False, oceancolor="LightBlue",
    #showlakes=True, lakecolor="Blue",
    #showrivers=True, rivercolor="Blue"
    center = {'lat': 30,'lon':0}
)

fig.update_geos(
    countrywidth = 0.1,countrycolor = "Black"
)
    

fig.write_image(FIGURES_DIR + 'restaurant_map.pdf')